In [ ]:
import os
import sys
import itertools
import numpy as np

import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

sys.path.append("/home/self_supervised_learning_gr/self_supervised_learning/dev/ProjetCassiopee")
from src.setup import setup_python, setup_pytorch
from src.transformation import RandomCrop, ObjectCrop
from src.dataset import RGBDObjectDataset, RGBDObjectDataset_Supervised_Contrast, RGBDObjectDataset_Unsupervised_Contrast

In [ ]:
setup_python()

In [ ]:
setup_pytorch(gpu=False)

In [ ]:
INPUT_SIZE = (256,256)
# TRANSFORMATION = transforms.Compose(
#         [transforms.ToTensor(),
#          transforms.Resize(size=INPUT_SIZE)])
TRANSFORMATION = transforms.Compose(
        [transforms.ToTensor()])
# CROP_TRANSFORMATION = RandomCrop(output_size=INPUT_SIZE,
#                                  offset_range=(-128, 0))
CROP_TRANSFORMATION = ObjectCrop(output_size=INPUT_SIZE,
                                 padding=(20,20),
                                 offset_range=(-10, 10))

# RGBDObjectDataset

In [ ]:
dataset = RGBDObjectDataset(path="rgbd-dataset",
                            mode="train",
                            class_names=["apple", "banana"],
                            modalities=["rgb", "depth", "mask", "loc"],
                            transformation=None,
                            crop_transformation=None,
                            nb_max_samples=20)

In [ ]:
len(dataset)

In [ ]:
print(dataset.class_dict)

In [ ]:
print(len(dataset.x))
print(dataset.x[:10])

In [ ]:
print(len(dataset.y))
print(dataset.y[:10])

In [ ]:
data_loader = DataLoader(dataset,
                         batch_size=2,
                         shuffle=False)

In [ ]:
len(data_loader)

In [ ]:
data_iter = iter(data_loader)

In [ ]:
for i, batch in enumerate(data_iter):
    # print(i)
    batch_rgb, batch_depth, batch_mask, batch_loc_x, batch_loc_y, batch_label = batch
    rgb, depth, mask, loc_x, loc_y, label = batch_rgb[0], batch_depth[0], batch_mask[0], batch_loc_x[0], batch_loc_y[0], batch_label[0]
    
    fig, axs = plt.subplots(1, 3, figsize=(12,3))
    axs[0].imshow(rgb.permute(1, 2, 0))
    axs[0].scatter([loc_x], [loc_y], marker="+", c="red")
    axs[1].imshow(depth.permute(1, 2, 0))
    axs[1].scatter([loc_x], [loc_y], marker="+", c="red")
    axs[2].imshow(mask.permute(1, 2, 0))
    axs[2].scatter([loc_x], [loc_y], marker="+", c="red")
    fig.suptitle(f"Label: {label}->{list(dataset.class_dict)[label]} | Location: ({loc_x},{loc_y})")
    
    if i == 4:
        break

In [ ]:
for i, batch in enumerate(itertools.islice(data_iter, len(data_loader))):
    # print(i)
    batch_rgb, batch_depth, batch_mask, batch_loc_x, batch_loc_y, batch_label = batch
    rgb, depth, mask, loc_x, loc_y, label = batch_rgb[0], batch_depth[0], batch_mask[0], batch_loc_x[0], batch_loc_y[0], batch_label[0]
    
    fig, axs = plt.subplots(1, 3, figsize=(12,3))
    axs[0].imshow(rgb.permute(1, 2, 0))
    axs[1].imshow(depth.permute(1, 2, 0))
    axs[2].imshow(mask.permute(1, 2, 0))
    fig.suptitle(f"Label: {label}->{list(dataset.class_dict)[label]} | Location: ({loc_x},{loc_y})")

In [ ]:
test = next(iter(data_loader))
print(type(test))
print(len(test))

In [ ]:
# Retrieve batch data
batch_rgb, batch_depth, batch_mask, batch_loc_x, batch_loc_y, batch_label = next(iter(data_loader))
print(type(batch_rgb))
print(type(batch_depth))
print(type(batch_mask))
print(type(batch_loc_x))
print(batch_loc_x.shape)
print(batch_loc_x)
print(type(batch_loc_y))
print(type(batch_label))
print(f"RGB batch shape: {batch_rgb.shape}")
print(f"Labels batch shape: {batch_label.shape}")

In [ ]:
# Retrieve individual data
rgb, depth, mask, loc_x, loc_y, label = batch_rgb[0], batch_depth[0], batch_mask[0], batch_loc_x[0], batch_loc_y[0], batch_label[0]
print(type(rgb))
print(type(depth))
print(type(mask))
print(type(loc_x))
print(loc_x.shape)
print(loc_x)
# loc_x = int(loc_x)
# print(type(loc_x))
# print(loc_x)
print(type(loc_y))
print(loc_y)
print(type(label))
print(label)
print(f"RGB shape: {rgb.shape}")
print(f"Labels shape: {label.shape}")

In [ ]:
print(rgb.size())
fig, axs = plt.subplots(1, 3, figsize=(12,3))
axs[0].imshow(rgb.permute(1, 2, 0))
axs[0].scatter([loc_x], [loc_y], marker="+", c="red")
axs[1].imshow(depth.permute(1, 2, 0))
axs[1].scatter([loc_x], [loc_y], marker="+", c="red")
axs[2].imshow(mask.permute(1, 2, 0))
axs[2].scatter([loc_x], [loc_y], marker="+", c="red")
fig.suptitle(f"Label: {label}->{list(dataset.class_dict)[label]} | Location: ({loc_x},{loc_y})")

## Crop transformation test

In [ ]:
crop_rgb, crop_depth, crop_mask, crop_loc_x, crop_loc_y = CROP_TRANSFORMATION(rgb, depth, mask, loc_x, loc_y)
print(crop_rgb.size())
fig, axs = plt.subplots(1, 3, figsize=(12,3))
axs[0].imshow(crop_rgb.permute(1, 2, 0))
axs[0].scatter([crop_loc_x], [crop_loc_y], marker="+", c="red")
axs[1].imshow(crop_depth.permute(1, 2, 0))
axs[1].scatter([crop_loc_x], [crop_loc_y], marker="+", c="red")
axs[2].imshow(crop_mask.permute(1, 2, 0))
axs[2].scatter([crop_loc_x], [crop_loc_y], marker="+", c="red")
fig.suptitle(f"Label: {label}->{list(dataset.class_dict)[label]} | Location: ({crop_loc_x},{crop_loc_y})")

# RGBDObjectDataset_Supervised_Contrastive

In [ ]:
dataset_supervised_contrast = RGBDObjectDataset_Supervised_Contrast(path="rgbd-dataset",
                                                                    mode="train",
                                                                    class_names=["apple", "banana"],
                                                                    modalities=["rgb", "depth", "mask", "loc"],
                                                                    transformation=TRANSFORMATION,
                                                                    crop_transformation=CROP_TRANSFORMATION,
                                                                    nb_max_samples=10)

In [ ]:
len(dataset_supervised_contrast)

In [ ]:
print(dataset_supervised_contrast.class_dict)

In [ ]:
print(len(dataset_supervised_contrast.x))
print(dataset_supervised_contrast.x[:10])

In [ ]:
print(len(dataset_supervised_contrast.y))
print(dataset_supervised_contrast.y[:10])

In [ ]:
data_loader_supervised_contrast = DataLoader(dataset_supervised_contrast,
                                             batch_size=10,
                                             shuffle=False)

In [ ]:
# Retrieve batch data
batch_p_data_1, batch_p_data_2, batch_n_data = next(iter(data_loader_supervised_contrast))
batch_p_rgb_1, batch_p_depth_1, batch_p_mask_1, batch_p_loc_x_1, batch_p_loc_y_1, batch_p_label_1 = batch_p_data_1
batch_p_rgb_2, batch_p_depth_2, batch_p_mask_2, batch_p_loc_x_2, batch_p_loc_y_2, batch_p_label_2 = batch_p_data_2
batch_n_rgb, batch_n_depth, batch_n_mask, batch_n_loc_x, batch_n_loc_y, batch_n_label = batch_n_data

In [ ]:
print(type(batch_p_rgb_1))
print(batch_p_rgb_1.shape)

In [ ]:
# Retrieve individual data
p_rgb_1, p_depth_1, p_mask_1, p_loc_x_1, p_loc_y_1, p_label_1 = batch_p_rgb_1[0], batch_p_depth_1[0], batch_p_mask_1[0], batch_p_loc_x_1[0], batch_p_loc_y_1[0], batch_p_label_1[0]
p_rgb_2, p_depth_2, p_mask_2, p_loc_x_2, p_loc_y_2, p_label_2 = batch_p_rgb_2[0], batch_p_depth_2[0], batch_p_mask_2[0], batch_p_loc_x_2[0], batch_p_loc_y_2[0], batch_p_label_2[0]
n_rgb, n_depth, n_mask, n_loc_x, n_loc_y, n_label = batch_n_rgb[0], batch_n_depth[0], batch_n_mask[0], batch_n_loc_x[0], batch_n_loc_y[0], batch_n_label[0]

In [ ]:
print(rgb.size())
fig, axs = plt.subplots(3, 3, figsize=(12,12))

axs[0,0].imshow(p_rgb_1.permute(1, 2, 0))
axs[0,0].scatter([p_loc_x_1], [p_loc_y_1], marker="+", c="red")
axs[0,1].imshow(p_depth_1.permute(1, 2, 0))
axs[0,1].scatter([p_loc_x_1], [p_loc_y_1], marker="+", c="red")
axs[0,2].imshow(p_mask_1.permute(1, 2, 0))
axs[0,2].scatter([p_loc_x_1], [p_loc_y_1], marker="+", c="red")
axs[0,0].set_title(f"Label: {p_label_1}->{list(dataset.class_dict)[p_label_1]} | Location: ({p_loc_x_1},{p_loc_y_1})")

axs[1,0].imshow(p_rgb_2.permute(1, 2, 0))
axs[1,0].scatter([p_loc_x_2], [p_loc_y_2], marker="+", c="red")
axs[1,1].imshow(p_depth_2.permute(1, 2, 0))
axs[1,1].scatter([p_loc_x_2], [p_loc_y_2], marker="+", c="red")
axs[1,2].imshow(p_mask_2.permute(1, 2, 0))
axs[1,2].scatter([p_loc_x_2], [p_loc_y_2], marker="+", c="red")
axs[1,0].set_title(f"Label: {p_label_2}->{list(dataset.class_dict)[p_label_2]} | Location: ({p_loc_x_2},{p_loc_y_2})")

axs[2,0].imshow(n_rgb.permute(1, 2, 0))
axs[2,0].scatter([n_loc_x], [n_loc_y], marker="+", c="red")
axs[2,1].imshow(n_depth.permute(1, 2, 0))
axs[2,1].scatter([n_loc_x], [n_loc_y], marker="+", c="red")
axs[2,2].imshow(n_mask.permute(1, 2, 0))
axs[2,2].scatter([n_loc_x], [n_loc_y], marker="+", c="red")
axs[2,0].set_title(f"Label: {n_label}->{list(dataset.class_dict)[n_label]} | Location: ({n_loc_x},{n_loc_y})")

# RGBDObject_Unsupervised_Contrast

In [ ]:
dataset_unsupervised_contrast = RGBDObjectDataset_Unsupervised_Contrast(path="rgbd-dataset",
                                                                        mode="train",
                                                                        class_names=["apple", "banana"],
                                                                        modalities=["rgb", "depth", "mask", "loc"],
                                                                        transformation=TRANSFORMATION,
                                                                        crop_transformation=CROP_TRANSFORMATION,
                                                                        nb_max_samples=10)

In [ ]:
len(dataset_unsupervised_contrast)

In [ ]:
print(dataset_unsupervised_contrast.class_dict)

In [ ]:
print(len(dataset_unsupervised_contrast.x))
print(dataset_unsupervised_contrast.x[:10])

In [ ]:
print(len(dataset_unsupervised_contrast.y))
print(dataset_unsupervised_contrast.y[:10])

In [ ]:
data_loader_unsupervised_contrast = DataLoader(dataset_unsupervised_contrast,
                                               batch_size=10,
                                               shuffle=False)

In [ ]:
# Retrieve batch data
batch_p_data_1, batch_p_data_2, batch_n_data = next(iter(data_loader_unsupervised_contrast))
batch_p_rgb_1, batch_p_depth_1, batch_p_mask_1, batch_p_loc_x_1, batch_p_loc_y_1, batch_p_label_1 = batch_p_data_1
batch_p_rgb_2, batch_p_depth_2, batch_p_mask_2, batch_p_loc_x_2, batch_p_loc_y_2, batch_p_label_2 = batch_p_data_2
batch_n_rgb, batch_n_depth, batch_n_mask, batch_n_loc_x, batch_n_loc_y, batch_n_label = batch_n_data

In [ ]:
print(type(batch_p_rgb_1))
print(batch_p_rgb_1.shape)

In [ ]:
# Retrieve individual data
p_rgb_1, p_depth_1, p_mask_1, p_loc_x_1, p_loc_y_1, p_label_1 = batch_p_rgb_1[0], batch_p_depth_1[0], batch_p_mask_1[0], batch_p_loc_x_1[0], batch_p_loc_y_1[0], batch_p_label_1[0]
p_rgb_2, p_depth_2, p_mask_2, p_loc_x_2, p_loc_y_2, p_label_2 = batch_p_rgb_2[0], batch_p_depth_2[0], batch_p_mask_2[0], batch_p_loc_x_2[0], batch_p_loc_y_2[0], batch_p_label_2[0]
n_rgb, n_depth, n_mask, n_loc_x, n_loc_y, n_label = batch_n_rgb[0], batch_n_depth[0], batch_n_mask[0], batch_n_loc_x[0], batch_n_loc_y[0], batch_n_label[0]

In [ ]:
print(rgb.size())
fig, axs = plt.subplots(3, 3, figsize=(12,12))

axs[0,0].imshow(p_rgb_1.permute(1, 2, 0))
axs[0,0].scatter([p_loc_x_1], [p_loc_y_1], marker="+", c="red")
axs[0,1].imshow(p_depth_1.permute(1, 2, 0))
axs[0,1].scatter([p_loc_x_1], [p_loc_y_1], marker="+", c="red")
axs[0,2].imshow(p_mask_1.permute(1, 2, 0))
axs[0,2].scatter([p_loc_x_1], [p_loc_y_1], marker="+", c="red")
axs[0,0].set_title(f"Label: {p_label_1}->{list(dataset.class_dict)[p_label_1]} | Location: ({p_loc_x_1},{p_loc_y_1})")

axs[1,0].imshow(p_rgb_2.permute(1, 2, 0))
axs[1,0].scatter([p_loc_x_2], [p_loc_y_2], marker="+", c="red")
axs[1,1].imshow(p_depth_2.permute(1, 2, 0))
axs[1,1].scatter([p_loc_x_2], [p_loc_y_2], marker="+", c="red")
axs[1,2].imshow(p_mask_2.permute(1, 2, 0))
axs[1,2].scatter([p_loc_x_2], [p_loc_y_2], marker="+", c="red")
axs[1,0].set_title(f"Label: {p_label_2}->{list(dataset.class_dict)[p_label_2]} | Location: ({p_loc_x_2},{p_loc_y_2})")

axs[2,0].imshow(n_rgb.permute(1, 2, 0))
axs[2,0].scatter([n_loc_x], [n_loc_y], marker="+", c="red")
axs[2,1].imshow(n_depth.permute(1, 2, 0))
axs[2,1].scatter([n_loc_x], [n_loc_y], marker="+", c="red")
axs[2,2].imshow(n_mask.permute(1, 2, 0))
axs[2,2].scatter([n_loc_x], [n_loc_y], marker="+", c="red")
axs[2,0].set_title(f"Label: {n_label}->{list(dataset.class_dict)[n_label]} | Location: ({n_loc_x},{n_loc_y})")